In [6]:
import pandas as pd
from sqlalchemy import create_engine

In [7]:
csv_file = "netflix_titles.csv"
titles_df = pd.read_csv(csv_file)
titles_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [8]:
titles_df["date_added"] = pd.to_datetime(titles_df['date_added'])
titles_df['year_added'] = titles_df['date_added'].dt.year
titles_df['month_added'] = titles_df['date_added'].dt.month
## dropping 'duration' column as its not necessary for this analysis

In [9]:
titles_df.drop(['director'], axis = 1, inplace = True)
titles_df.drop(['cast'], axis = 1, inplace = True)

In [10]:
titles_df.head()

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
1,s2,TV Show,Blood & Water,South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0
2,s3,TV Show,Ganglands,NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0
4,s5,TV Show,Kota Factory,India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0


In [11]:
titles_df.drop(['description'], axis = 1, inplace = True)

In [12]:
print(titles_df['country'].value_counts())

United States                             2818
India                                      972
United Kingdom                             419
Japan                                      245
South Korea                                199
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: country, Length: 748, dtype: int64


In [13]:
countries_expanded = titles_df['country'].str.split(',', expand=True)
countries_expanded.columns = ['Country'+str(i) for i in countries_expanded.columns]

countries_expanded_concat = pd.concat([titles_df,countries_expanded], axis=1)

year_country_produced_df = pd.melt(countries_expanded_concat, id_vars=['show_id','release_year','rating'], value_vars=countries_expanded.columns, var_name='Country Number', value_name='Country Produced').dropna()

In [33]:
year_country_produced_df.head()

,show_id,release_year,Country Number,Country Produced
0,s1,2020,Country0,United States
1,s2,2021,Country0,South Africa
4,s5,2021,Country0,India
7,s8,1993,Country0,United States
8,s9,2021,Country0,United Kingdom


In [14]:
print(year_country_produced_df['Country Produced'].value_counts())

United States     3211
India             1008
United Kingdom     628
 United States     479
Canada             271
                  ... 
 Sri Lanka           1
Mozambique           1
 Palestine           1
 Cuba                1
 Mongolia            1
Name: Country Produced, Length: 197, dtype: int64


In [15]:
year_country_produced_df

,show_id,release_year,rating,Country Number,Country Produced
0,s1,2020,PG-13,Country0,United States
1,s2,2021,TV-MA,Country0,South Africa
4,s5,2021,TV-MA,Country0,India
7,s8,1993,TV-MA,Country0,United States
8,s9,2021,TV-14,Country0,United Kingdom
...,...,...,...,...,...
78859,s8404,2014,PG-13,Country8,Germany
85496,s6234,2017,TV-MA,Country9,Sweden
87666,s8404,2014,PG-13,Country9,Netherlands
94303,s6234,2017,TV-MA,Country10,United States


In [16]:
engine = create_engine('sqlite:///year_country_produced.db', echo=True)
sqlite_connection = engine.connect()

In [17]:
sqlite_table = "Country Production"
year_country_produced_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-10-31 18:00:06,581 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Country Production")
2021-10-31 18:00:06,582 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-31 18:00:06,583 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Country Production")
2021-10-31 18:00:06,584 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-31 18:00:06,586 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Country Production" (
	"index" BIGINT, 
	show_id TEXT, 
	release_year BIGINT, 
	rating TEXT, 
	"Country Number" TEXT, 
	"Country Produced" TEXT
)


2021-10-31 18:00:06,587 INFO sqlalchemy.engine.Engine [no key 0.00070s] ()
2021-10-31 18:00:06,592 INFO sqlalchemy.engine.Engine COMMIT
2021-10-31 18:00:06,593 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Country Production_index" ON "Country Production" ("index")
2021-10-31 18:00:06,594 INFO sqlalchemy.engine.Engine [no key 0.00089s] ()
2021-10-31 18:00:06,599 INFO sqlalchemy.engine.Engine COMMIT
2021-10-31 18:00:06,604 INFO sqlalchemy.engine.E

In [18]:
sqlite_connection.close()

In [19]:
database_path = "year_country_produced_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

In [52]:
year_country_produced_df.to_sql(name='ProductionCompany', con=engine, if_exists='append', index=False)

In [53]:
engine.table_names()

C:\Users\jimmy\AppData\Local\Temp/ipykernel_13340/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['ProductionCompany']

In [54]:
pd.read_sql_query('select * from ProductionCompany limit 50', con=engine).head()

,show_id,release_year,Country Number,Country Produced
0,s1,2020,Country0,United States
1,s2,2021,Country0,South Africa
2,s5,2021,Country0,India
3,s8,1993,Country0,United States
4,s9,2021,Country0,United Kingdom
